# Løsninger for oppgaver i kurs i imputering

Datasettet som skal brukes for oppgaver i dette kurset er avtale årsverk fyioterapeauter i kommunen.
Datasettet heter fysio og er hentet fra statistikkbanken.
Det blir publisert både foreløpige tall 15. mars og endelige tall 15. juni. 
Når det publiseres 15. mars er det ca 5 prosent frafall og dette trenger vi å imputere for å lage
Landstall og andre aggregater. 

## Variabler datasett fysio
* 'kommnr' - Nummer for kommune
* 'Kommune' - Navn og nummer for kummune 
* 'arsverk_2020' - Avtale årsverk fysiterapeuter 2020
* 'arsverk_2021_for' Avtale årsverk fysiterapeuter 2021, 15. mars foreløpige tall
* 'Brutto_driftsutgifter_helse_2021' - Driftsutgifter for helse i 2021
* 'Folkemengde_2021' - folkemengden 1.1.2021
* 'arsverk_2021_end' - Avtale årsverk fysiterapeuter 2021, 15. juni endelige  tall
* 'kostragr' - Kostragruppe
* 'kostranavn' - Navn på kostragruppe
* 'kommnavn' - kommue navn
*  'imp' - 1 ikke imputert, 2 imputert



In [ ]:
#Pakker som blir brukt i kurset
library(tidyverse) # språk for koding
library(plotly) # Interaktivt figurer
library(dcmodify) # regelimputering
library(simputation) # imputeringsmetoder
library(lumberjack) #  logging av endringer

# Oppgave 1. Frafall og historisk imputering
### 1a) Les inn datasett fysio og beregn frafallet i forløpige tall?
* Først les inn datasettet fysio.scv. det er kommaseparert med punktum som desimaltegn. 
  (Hint read_csv2, pga en bug i pakken simputation som vi skal bruke senere omgjør datasettet til dataframe as.data.frame)
* Sjekk at du har fått lest inn datasettet riktig. 
  (hint head)
* Finn antall observasjoner og missing og beregn frafallsprosenten for variabelen arsverk_2021_for


In [ ]:
# Leser inn datasettet, En funksjon vi skal bruke trenger at datasettet er dataframe
fysio <- as.data.frame(read_csv2("fysio.csv"))

head(fysio)

tabell <- fysio %>%
  summarise(ant_mis = sum(is.na(arsverk_2021_for)), tot = n()) %>%
  mutate(frafall=ant_mis*100/tot)
tabell



### 1b) Hva er konsekvensene av å ignorere frafallet? 
* Hint beregn sum av årsverk 2020, foreløpig og endelige tall 2021


In [ ]:
fysio %>% 
  summarise(tot_2020=sum(arsverk_2020),tot_2020f=sum(arsverk_2021_for, na.rm=TRUE), tot_2021e=sum(fysio$arsverk_2021_end))


### 1c) Imputer frafallet i foreløpige tall med forrige års verdi. Bruk pakken dcmodify.
* Hvor god sammenheng er det mellom to årganger av årsverk fysioterapeuter?
* Imputer frafallet med historisk imputering
* Hva blir totalen nå?
* Hvor stor blir feilen - RMSE? sammenlign med endelige tall
* Bruk grafikk til å vurder hvor god metoden er. Lag et scatterplot for å sammenligne foreløpige og endelige tall

In [ ]:
# Beregner Korrelasjon  
fysio %>% summarize (kor=cor(arsverk_2020, arsverk_2021_end))
 

# Setter opp regelretting                              
modi <- modifier(if(is.na(arsverk_2021_for) ) arsverk_2021_for <- arsverk_2020) 
              
# Kjører regelretting 
fysio_1 <- modify(fysio, modi)

# Finner totalen ved å summere
tot_2021imp <- fysio_1 %>% summarise(sum(arsverk_2021_for))
tot_2021imp

# Beregner feilen
res <-fysio_1 %>% mutate(feil=(arsverk_2021_for-arsverk_2021_end)^2) %>%
         summarise(mse=sum(feil), n=n()) %>%
         mutate(rmse=sqrt(mse/n))

rmse1<-round(res[3],3)
rmse1
 
# Lager grafikk                 
fig1 <- fysio_1 %>% filter(kostragr=="EKG06") %>%

       plot_ly( x = ~arsverk_2021_end, y= ~arsverk_2021_for,
        type ="scatter",mode ="markers", split = ~imp,
        text = ~Kommune ,       
        hovertemplate = paste(
      "<b>%{text}</b><br><br>",
      "%{yaxis.title.text}: %{y:}<br>",
      "%{xaxis.title.text}: %{x:}<br>",
      "<extra></extra>"
      ) ) %>%
       layout(title = "Historisk imputering, kostragruppe 6", xaxis = list(title = "Endelige tall"),
        yaxis = list(title = "Foreløpige tall"), legend=list(title=list(text='Outlier:')))
fig1  

                

# Oppgave 2 testing av imputeringsmetoder


### 2a Imputer med gjennomsnittet innen hver kostragruppe og vurder resultatet. Bruk pakken simputation. 
*     Hva blir totalen nå?
*     Hvor stor blir feilen - RMSE? sammenlign med endelige tall
*     Bruk grafikk til å vurder hvor god metoden er i kostragruppe 6. Lag et scatterplot for å sammenligne foreløpige og endelige tall



In [ ]:

fysio_2 <- fysio %>% impute_proxy(arsverk_2021_for ~ mean(arsverk_2020, na.rm = TRUE)| kostragr)

# Finner totalen ved å summere
tot_2021imp2 <- fysio_2 %>% summarise(sum(arsverk_2021_for))
tot_2021imp2

# Beregner feilen rmse
res <-fysio_2 %>% mutate(feil=(arsverk_2021_for-arsverk_2021_end)^2) %>%
         summarise(mse=sum(feil), n=n()) %>%
         mutate(rmse=sqrt(mse/n))

rmse2 <- round(res[3],3)
rmse2

# Lager en figur
fig2 <- fysio_2 %>% filter(kostragr=="EKG06") %>%

       plot_ly( x = ~arsverk_2021_end, y= ~arsverk_2021_for,
        type ="scatter",mode ="markers", split = ~imp,
        text = ~Kommune ,       
        hovertemplate = paste(
      "<b>%{text}</b><br><br>",
      "%{yaxis.title.text}: %{y:.1f}<br>",
      "%{xaxis.title.text}: %{x:}<br>",
      "<extra></extra>"
      ) ) %>%
       layout(title = "Gjennomsnittsimputering, kostragruppe 6", xaxis = list(title = "Endelige tall"),
        yaxis = list(title = "Foreløpige tall"), legend=list(title=list(text='Outlier:')))
fig2  




### 2b Imputer med random hotdeck innen hver kostragruppe.
*     Set en seed for tilfeldig trekking set.seed() 
*     Kjør Hotdeck imputering med bruk pool= "complete"
*     Hva blir totalen nå?
*     Hvor stor blir feilen - RMSE? sammenlign med endelige tall
*     Bruk grafikk til å vurder hvor god metoden er i kostra gruppe 6. Lag et scatterplot for å sammenligne foreløpige og endelige tall

In [ ]:
set.seed(123456789)
fysio_3 <- fysio %>% impute_rhd(arsverk_2021_for ~ 1| kostragr , pool = "complete" )

# Finner totalen ved å summere
tot_2021imp3 <- fysio_3 %>% summarise(sum(arsverk_2021_for))
tot_2021imp3

# Beregner feilen rmse
res <-fysio_3 %>% mutate(feil=(arsverk_2021_for-arsverk_2021_end)^2) %>%
         summarise(mse=sum(feil), n=n()) %>%
         mutate(rmse=sqrt(mse/n))

rmse3 <- round(res[3],3)
rmse3

# Lager en figur
fig3 <- fysio_3 %>% filter(kostragr=="EKG06") %>%

       plot_ly( x = ~arsverk_2021_end, y= ~arsverk_2021_for,
        type ="scatter",mode ="markers", split = ~imp,
        text = ~Kommune ,       
        hovertemplate = paste(
      "<b>%{text}</b><br><br>",
      "%{yaxis.title.text}: %{y:.1f}<br>",
      "%{xaxis.title.text}: %{x:}<br>",
      "<extra></extra>"
      ) ) %>%
       layout(title = "Random hotdeck imputering, kostragruppe 6", xaxis = list(title = "Endelige tall"),
        yaxis = list(title = "Foreløpige tall"), legend=list(title=list(text='Outlier:')))
fig3  




### 2c Imputer med nærmeste nabo imputering innen hver kostragruppe.
*     Velg hvilke variabler nærmeste nabo skal beregnes fra og kjør imputering med k=1
*     Hva blir totalen nå?
*     Hvor stor blir feilen - RMSE? sammenlign med endelige tall
*     Bruk grafikk til å vurder hvor god metoden er i kostragruppe 6. Lag et scatterplot for å sammenligne foreløpige og endelige tall

In [ ]:
fysio_4 <- fysio %>% impute_knn(arsverk_2021_for ~ Brutto_driftsutgifter_helse_2021 + Folkemengde_2021,  k=1)
# Finner totalen ved å summere
tot_2021imp4 <- fysio_4 %>% summarise(sum(arsverk_2021_for))
tot_2021imp4

# Beregner feilen rmse 
res <-fysio_4 %>% mutate(feil = (arsverk_2021_for-arsverk_2021_end)^2) %>%
         summarise(mse=sum(feil), n=n()) %>%
         mutate(rmse=sqrt(mse/n))

rmse4 <- round(res[3],3)
rmse4

# Lager en figur
fig4 <- fysio_4 %>% filter(kostragr=="EKG06") %>%

       plot_ly( x = ~arsverk_2021_end, y= ~arsverk_2021_for,
        type ="scatter",mode ="markers", split = ~imp,
        text = ~Kommune ,       
        hovertemplate = paste(
      "<b>%{text}</b><br><br>",
      "%{yaxis.title.text}: %{y:.1f}<br>",
      "%{xaxis.title.text}: %{x:}<br>",
      "<extra></extra>"
      ) ) %>%
       layout(title = "Nærmeste nabo imputering, kostragruppe 6", xaxis = list(title = "Endelige tall"),
        yaxis = list(title = "Foreløpige tall"), legend=list(title=list(text='Outlier:')))
fig4  


### 2d Imputer med regresjonsimputering innen hver kostragruppe.
*     Velg variabler som forklaringsvariabler
*     Test gjerne om du har en god modellen med bruk av vanlig regresjonfunksjon (lm) og grafikk
*     Hva blir totalen nå?
*     Hvor stor blir feilen - RMSE? sammenlign med endelige tall
*     Bruk grafikk til å vurder hvor god metoden er. Lag et scatterplot for å sammenligne foreløpige og endelige tall

In [ ]:
# Test av modell
res <- lm(data=fysio, arsverk_2021_for ~Folkemengde_2021)
summary(res)
plot_reg <-fysio %>% filter(!is.na(arsverk_2021_for)) %>% mutate(pred=predict(res)) %>%
  plot_ly(x=~Folkemengde_2021, y=~arsverk_2021_for, type ="scatter",mode ="markers") %>%
     add_trace(y =~pred , mode = 'lines') 
plot_reg


# Kjører imputering med regresjon
fysio_5 <- fysio %>% impute_lm(arsverk_2021_for ~Folkemengde_2021)
# Finner totalen ved å summere
tot_2021imp5<- fysio_5 %>% summarise(sum(arsverk_2021_for))
tot_2021imp5

# Beregner feilen rmse
res <-fysio_5 %>% mutate(feil=(arsverk_2021_for-arsverk_2021_end)^2) %>%
         summarise(mse=sum(feil), n=n()) %>%
         mutate(rmse=sqrt(mse/n))

rmse5 <- round(res[3],3)
rmse5

# Lager en figur
fig5 <- fysio_5 %>% filter(kostragr=="EKG06") %>%

       plot_ly( x = ~arsverk_2021_end, y= ~arsverk_2021_for,
        type ="scatter",mode ="markers", split = ~imp,
        text = ~Kommune ,       
        hovertemplate = paste(
      "<b>%{text}</b><br><br>",
      "%{yaxis.title.text}: %{y:.1f}<br>",
      "%{xaxis.title.text}: %{x:}<br>",
      "<extra></extra>"
      ) ) %>%
       layout(title = "Regresjons imputering, kostragruppe 6", xaxis = list(title = "Endelige tall"),
        yaxis = list(title = "Foreløpige tall"), legend=list(title=list(text='Outlier:')))
fig5 

### 2e Imputer med prediktiv mean matching innen hver kostragruppe.
*     Velg variabler som forklaringsvariabler og kjør imputering
*     Hva blir totalen nå?
*     Hvor stor blir feilen - RMSE? sammenlign med endelige tall
*     Hvordan ser det ut grafisk for kostra gruppe 6? sammenlign med endelige tal

In [ ]:
fysio_6 <- fysio %>% impute_pmm(arsverk_2021_for ~ arsverk_2020)
# Finner totalen ved å summere
tot_2021imp6<- fysio_6 %>% summarise(sum(arsverk_2021_for))
tot_2021imp6

# Beregner feilen rmse
res <-fysio_6 %>% mutate(feil=(arsverk_2021_for-arsverk_2021_end)^2) %>%
         summarise(mse=sum(feil), n=n()) %>%
         mutate(rmse=sqrt(mse/n))

rmse6 <- round(res[3],3)
rmse6

# Lager en figur
fig6 <- fysio_6 %>% filter(kostragr=="EKG06") %>%

       plot_ly( x = ~arsverk_2021_end, y= ~arsverk_2021_for,
        type ="scatter",mode ="markers", split = ~imp,
        text = ~Kommune ,       
        hovertemplate = paste(
      "<b>%{text}</b><br><br>",
      "%{yaxis.title.text}: %{y:.1f}<br>",
      "%{xaxis.title.text}: %{x:}<br>",
      "<extra></extra>"
      ) ) %>%
       layout(title = "Predictiv mean matching pmm imputering, kostragruppe 6", xaxis = list(title = "Endelige tall"),
        yaxis = list(title = "Foreløpige tall"), legend=list(title=list(text='Outlier:')))
fig6 

# Oppgave 3. Velg endelig modell for imputering og sett opp logging av endring av verdier og total



In [ ]:
# Sammenligner feilen for de forkjellige metodene
sammen <- c(rmse1,rmse2,rmse3,rmse4,rmse5,rmse6)
names(sammen)<-c("Historisk", "Gjennomsnitt", "Random_Hotdeck","Nærmeste_nabo", "Regresjon", "PMM")
sammen

# Velger metode og setter opp logging
fysio_imp <- fysio %>>%
  start_log(cellwise$new(key = "kommnr")) %>>%
  start_log(expression_logger$new(total = sum(arsverk_2021_for, na.rm = TRUE), count_tom = sum(is.na(arsverk_2021_for)))) %>>%

   impute_lm(arsverk_2021_for ~ arsverk_2020) %>>%
  
  dump_log("cellwise", file = "log_cellwise.csv") %>>%
  dump_log("expression_logger", file = "log_expr.csv")
  
offer_3_1_celllwise <- read.csv("log_cellwise.csv")
offer_3_1_expr <- read.csv("log_expr.csv")

offer_3_1_celllwise
offer_3_1_expr